<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/729_RGOv2_DataScope.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


Now we anchor strategy to reality.

We will map:

* Your existing CSV structure
* To V2 derived intelligence
* Without adding unnecessary columns

This is where architectural discipline matters.

---

# 📂 Step 1 — Understand the Raw Layers

You currently have:

1. `retail_customers.csv`
2. `retail_weekly_sales.csv`
3. `stock_availability.csv`

We will treat them as:

* Static customer attributes
* Time-series revenue layer
* Time-series operational layer

---

# 🧱 Layer 1: retail_customers.csv

### Likely Columns (Typical Structure)

```
customer_id
customer_name
region
tier
signup_date
baseline_revenue
lifetime_value
```

### V2 Usage Mapping

| Raw Column       | Used For            |
| ---------------- | ------------------- |
| customer_id      | Primary key         |
| region           | Segment aggregation |
| tier             | Tier multiplier     |
| lifetime_value   | Optional validation |
| baseline_revenue | BaseGap calculation |

We do NOT need to change this dataset.

We will derive:

```
customer_value_multiplier (from tier)
```

No new columns required.

---

# 📊 Layer 2: retail_weekly_sales.csv

### Likely Columns

```
customer_id
week
revenue
product_category
units_sold
```

### V2 Derived Fields

From this file we compute:

---

## 🟢 Revenue Gap Inputs

```
current_revenue
baseline_revenue
gap_amount
decline_percent
```

---

## 🟡 Structural Signals

Derived from rolling window:

```
consecutive_decline_weeks
consecutive_zero_spend_weeks
decline_velocity_percent
revenue_volatility
```

These are NOT stored in CSV.
They are computed inside the orchestrator.

---

## 🔵 Recurrence Layer (New V2 Concept)

We introduce lightweight tracking:

```
prior_gap_flag
prior_structural_flag
```

This will eventually come from:

* Historical snapshot JSON (not CSV)
* Or state tracking

No CSV modification required yet.

---

# 🏭 Layer 3: stock_availability.csv

### Likely Columns

```
week
product_category
stock_available
customer_id (optional)
```

### V2 Derived Fields

We calculate:

```
stockout_flag
stockout_overlap_weeks
percent_gap_due_to_stockout
```

These are computed by:

1. Matching week
2. Matching product category
3. Comparing revenue drop to stockout presence

Again:
No new CSV columns required.

---

# 🧮 Where Each V2 Feature Comes From

| V2 Feature        | Source                  |
| ----------------- | ----------------------- |
| BaseGap           | weekly_sales + baseline |
| Decline Streak    | weekly_sales            |
| Zero Spend Streak | weekly_sales            |
| Velocity          | weekly_sales            |
| Volatility        | weekly_sales            |
| Stockout Overlap  | stock_availability      |
| Tier Multiplier   | customers               |
| Structural Tier   | derived logic           |
| Root Cause        | derived logic           |
| Exposure Index    | derived logic           |

Everything comes from existing data.

This is excellent.

---

# 📦 Do We Need New Raw Data?

No.

But we *may* need:

1. Clear baseline definition
2. Clear trailing window size (e.g., 4 weeks)
3. Clear week ordering
4. Confirmed tier mapping

That’s it.

---

# 🔎 Critical Questions Before Implementation

Now we must confirm:

### 1️⃣ How is baseline_revenue defined?

* Fixed value in customers table?
* Rolling 8-week average?
* First 12 weeks average?
* Configurable?

My recommendation:

Use rolling trailing average (configurable window).

More realistic.
More adaptive.
Still deterministic.

---

### 2️⃣ How many weeks of data exist?

Structural logic needs:

Minimum 6–8 weeks of history.

Do you have that?

---

### 3️⃣ Does stock_availability map to customer OR product?

If product-level only:
We must estimate customer impact via:

```
customer_product_mix × stockout_weeks
```

That’s fine.
But we must know structure.

---

# 🏗 Clean Mapping Summary

We will NOT:

* Add new CSVs
* Add 30 new columns
* Change schema unnecessarily

We WILL:

* Compute structural metrics inside orchestrator
* Compute stockout attribution dynamically
* Compute REI dynamically
* Persist historical snapshots separately (JSON later)





# ✅ Official V2 Data Assumptions (Locked)

## 1️⃣ Baseline Definition → Rolling Trailing Average

We will define:

```
baseline_revenue = rolling_mean(revenue, window = N weeks)
```

### Config:

```
baseline_window_weeks = 6
```

Why 6?

* Long enough to smooth noise
* Short enough to react to shifts
* CFO-defensible
* Retail-reasonable
* Not overfit

This makes baseline adaptive.

No static baseline in customers table.

---

## 2️⃣ Structural Window Size

Structural signals will use:

```
structural_window_weeks = 8
```

Why 8?

* Enough history for volatility calculation
* Enough time to detect patterns
* Not too heavy computationally
* Clean rolling window

---

## 3️⃣ Stock Mapping Granularity → Product-Level Attribution

We assume:

* stock_availability.csv is product-level by week
* sales include product_category

We will attribute operational leakage by:

```
If product_category stockout in week
AND customer revenue for that category drops
→ operational attribution
```

If stockout dataset is not customer-specific, that’s fine.

We model impact probabilistically but deterministically:

```
stockout_overlap_weeks
percent_gap_due_to_stockout
```

Still rule-based.
Still explainable.

---

# 🏗 Updated Data Architecture (Now Fully Concrete)

## Raw Layer

Customers
Weekly Sales
Stock Availability

---

## Derived Layer

### Revenue Layer

* rolling_baseline
* gap_amount
* decline_percent

### Structural Layer

* consecutive_decline_weeks
* consecutive_zero_spend_weeks
* velocity_percent
* volatility_score
* structural_score
* structural_tier

### Operational Layer

* stockout_overlap_weeks
* stockout_attribution_percent
* operational_flag

### Exposure Layer

* churn_multiplier
* structural_multiplier
* tier_multiplier
* recurrence_multiplier
* REI

### Classification Layer

* root_cause

---

Everything now has a source.
Nothing is ambiguous.
Nothing is speculative.

---

# 🧠 Why This Is Enterprise-Strong

* Baseline adaptive (not static)
* Structural signals pattern-based
* Operational attribution deterministic
* Exposure weighted but capped
* Entire system config-driven
* Fully auditable

This is how serious enterprise systems are designed.


